## Life Cycle of Data Science 

1. Data Analysis
2. Feature Engineering
3. Feature Selection
4. Model Building
5. Model Deployment

1. 'battery_power' -battery_power
2. 'blue' -bluetooth
3. 'clock_speed' -clock_speed
4. 'dual_sim' -sim
5. 'fc' - Front Screen
6. 'four_g'- 4G Connection
7. 'int_memory'- Internal Memory
8. 'm_dep'- Slimness
9. 'mobile_wt' -Weight
10. 'n_cores'- Processor
11. 'pc'- Primary camera
12. 'px_height' -Pixel hight
13. 'px_width' - Pixel Width
14. 'ram' - ram size
15. 'sc_h'- screen hight
16. 'sc_w'- screen width
17. 'talk_time'- talk time
18. 'three_g'- 3G
19. 'touch_screen'- Touch facility
20. 'wifi' - wifi
21. 'price_range'- price

In [ ]:
## Data Analysis
## Main Aim is to understand about the data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.pandas.set_option('display.max_columns',None)

In [ ]:
dataset = pd.read_csv('../input/mobile-price-classification/train.csv')
print(dataset.shape)
dataset.head()

In [ ]:
dataset.info()

we are having 21 columns all are 
###### Numerical values 
######  Having NO Null values

In [ ]:
## Since we are going to predict price, lets analyse based on price_range 

dataset['price_range'].value_counts()

we can see the values is not numerical, it is classified into four categories having count of 500 each

In [ ]:
sns.countplot(dataset['price_range'])
plt.xlabel('price_range')


lets count the unique values of all columns to find the discreate numerical variable

In [ ]:
for i in dataset.columns:
    if len(dataset[i].unique()) < 100 and i != 'price_range' :
        print(i,'-----', len(dataset[i].unique()))
    

here we can see that we are having 15 discreate numerical variable 

In [ ]:
###lets plot fo these discreate variables
discreate_numerical = []
for i in dataset.columns:
    if len(dataset[i].unique()) < 50:
        discreate_numerical.append(i)

        

In [ ]:
plt.figure(figsize=(25,50))
for i in enumerate(discreate_numerical):
    plt.subplot(8,2,i[0]+1)
    sns.barplot(dataset[i[1]],dataset['price_range'])
    plt.xlabel(i[1])

here we can see most of the features are categorised in numbers

In [ ]:
plt.figure(figsize=(18,36))
for i in enumerate(discreate_numerical):
    plt.subplot(8,2,i[0]+1)
    sns.boxplot(dataset[i[1]],dataset['price_range'])
    plt.xlabel(i[1])

we can see from the above the box plot for every features varies huge this indicates most of the features are providing information for result

Lets do analysis for Numerical features

In [ ]:
numerical_fet = []

for i in dataset.columns:
    if i  not in discreate_numerical:
        numerical_fet.append(i)


In [ ]:
plt.figure(figsize=(18,36))
for i in enumerate(numerical_fet):
    plt.subplot(8,2,i[0]+1)
    sns.histplot(dataset[i[1]],kde=True)
    plt.xlabel(i[1])

Here we can see only px_height shows a right skewed distribution

###### Lets analyse the corelation between these pics

In [ ]:
plt.figure(figsize=(25,25))
sns.heatmap(dataset.corr(),annot=True)

 We can see here that Ram is highly co related with Price

Since most of the values independent of each other lets get to feature selection

In [ ]:
y = dataset['price_range']
dataset.drop(['price_range'],axis=1,inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix

x_tr,x_te,y_tr,y_te = train_test_split(dataset,y,test_size=0.3,random_state=42)

cla = RandomForestClassifier()
cla.fit(x_tr,y_tr)
acuracy = accuracy_score(y_te,cla.predict(x_te))
cm =confusion_matrix(y_te,cla.predict(x_te))
print(acuracy)
sns.heatmap(cm,annot=True)

Here we are getting accuracy of 0.865 and we can also see that many features affect the prediction

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
selc_ten = SelectKBest(chi2,k=10).fit(x_tr,y_tr)

In [ ]:
print(selc_ten.scores_)
for i in range(len(dataset.columns)):
    print(dataset.columns[i],' :    ',selc_ten.scores_[i] )

selecting most important features from the list based on their value

battery_power,ram,mobile_wt,px_height,px_width,fc,pc,sc_w,talk_time,int_memory

the above 10 features affect the model most 

In [ ]:
X_tr = selc_ten.transform(x_tr)
X_te = selc_ten.transform(x_te)

In [ ]:
clf_2 = RandomForestClassifier()

In [ ]:
clf_2.fit(x_tr,y_tr)

In [ ]:
accuracy_score(y_te,clf_2.predict(x_te))

here im getting accuracy .875 increased when compared to preivious

In [ ]:
cm1 = confusion_matrix(y_te,clf_2.predict(x_te))
sns.heatmap(cm1,annot=  True)

from the above model we can see that unwanted features giving us  type 1 and type 2 errors are minimizsed 

Lets see wat can we do if we want to increase more



#### Recursive feature elimination with cross validation and random forest classification
In previous method we found how many features we needed most by our own choice,

But in this method we can find how many features can give best accuracy and the choice is made by cross validation

In [ ]:
from sklearn.feature_selection import RFECV

clf_3 = RandomForestClassifier()
rfecv = RFECV(estimator=clf_3,cv=5, step=1,scoring='accuracy')
rfecv.fit(x_tr,y_tr)

In [ ]:
print(rfecv.n_features_)

here it took 5 features to make the model to be optimized

In [ ]:
print(x_tr.columns[rfecv.support_])

In [ ]:
plt.figure()
plt.xlabel('features')
plt.ylabel('accuracy')
plt.plot(range(1,len(rfecv.grid_scores_)+1),rfecv.grid_scores_)


here we can see at 5 we are getting maximum accuracy, so we will be choosing 5 features for prediction

###### Now lets predict for test data

###### Recursive feature elimination with cross validation and random forest classification